In [22]:
from langgraph_sdk import get_client

client = get_client(url="http://127.0.0.1:2024")

In [23]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [24]:
initial_input = {"messages": "Multiply 2 and 3"}
thread = await client.threads.create()

In [25]:
# Search all hosted graphs
assistants = await client.assistants.search()
assistants

[{'assistant_id': '6f6fce9a-b777-529d-9699-dd340ddec86c',
  'graph_id': 'dynamic_breakpoints',
  'config': {},
  'metadata': {'created_by': 'system'},
  'name': 'dynamic_breakpoints',
  'created_at': '2025-05-25T13:10:51.548011+00:00',
  'updated_at': '2025-05-25T13:10:51.548011+00:00',
  'version': 1,
  'description': None},
 {'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
  'graph_id': 'agent',
  'config': {},
  'metadata': {'created_by': 'system'},
  'name': 'agent',
  'created_at': '2025-05-25T13:10:51.542586+00:00',
  'updated_at': '2025-05-25T13:10:51.542586+00:00',
  'version': 1,
  'description': None}]

In [27]:
async for event in client.runs.stream(thread_id=thread["thread_id"], assistant_id="agent",input=initial_input, stream_mode="values", interrupt_before=["assistant"]):
    print(f"Receiving new event of type: {event.event}...")
    messages = event.data.get('messages', [])
    if messages:
        print(messages[-1])
    print("-" * 50)

Receiving new event of type: metadata...
--------------------------------------------------
Receiving new event of type: values...
{'content': 'Multiply 2 and 3', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'b4fde500-ab20-4797-8577-632061bdd1a3', 'example': False}
--------------------------------------------------


In [30]:
current_state = await client.threads.get_state(thread_id=thread["thread_id"])
current_state

{'values': {'messages': [{'content': 'Multiply 2 and 3',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': 'b4fde500-ab20-4797-8577-632061bdd1a3',
    'example': False}]},
 'next': ['assistant'],
 'tasks': [{'id': '3d531a85-8d84-b581-eccb-bf0646d17bc5',
   'name': 'assistant',
   'path': ['__pregel_pull', 'assistant'],
   'error': None,
   'interrupts': [],
   'checkpoint': None,
   'state': None,
   'result': None}],
 'metadata': {'user-agent': 'langgraph-sdk-py/0.1.70',
  'x-request-id': '5994aae3-b3ba-4907-a86c-f2b088eec277',
  'langgraph_auth_user': None,
  'langgraph_auth_user_id': '',
  'langgraph_auth_permissions': [],
  'langgraph_request_id': '5994aae3-b3ba-4907-a86c-f2b088eec277',
  'graph_id': 'agent',
  'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
  'user_id': '',
  'created_by': 'system',
  'run_attempt': 1,
  'langgraph_version': '0.4.7',
  'langgraph_plan': 'developer',
  'langgraph_host': 'self-hosted',

In [31]:
last_message = current_state['values']['messages'][-1]
last_message

{'content': 'Multiply 2 and 3',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'human',
 'name': None,
 'id': 'b4fde500-ab20-4797-8577-632061bdd1a3',
 'example': False}

In [32]:
last_message['content'] = "No, actually multiply 3 and 3!"
last_message

{'content': 'No, actually multiply 3 and 3!',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'human',
 'name': None,
 'id': 'b4fde500-ab20-4797-8577-632061bdd1a3',
 'example': False}

In [33]:
last_message

{'content': 'No, actually multiply 3 and 3!',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'human',
 'name': None,
 'id': 'b4fde500-ab20-4797-8577-632061bdd1a3',
 'example': False}

In [34]:
await client.threads.update_state(thread_id=thread['thread_id'], values={"messages": last_message})

{'checkpoint': {'thread_id': 'b19128a7-110c-465c-82bd-2e00b5f969d0',
  'checkpoint_ns': '',
  'checkpoint_id': '1f039f94-695b-6fcf-8001-59b62e0f5fa0'},
 'configurable': {'thread_id': 'b19128a7-110c-465c-82bd-2e00b5f969d0',
  'checkpoint_ns': '',
  'checkpoint_id': '1f039f94-695b-6fcf-8001-59b62e0f5fa0'},
 'checkpoint_id': '1f039f94-695b-6fcf-8001-59b62e0f5fa0'}

In [35]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input=None,
    stream_mode="values",
    interrupt_before=["assistant"],
):
    print(f"Receiving new event of type: {chunk.event}...")
    messages = chunk.data.get('messages', [])
    if messages:
        print(messages[-1])
    print("-" * 50)

Receiving new event of type: metadata...
--------------------------------------------------
Receiving new event of type: values...
{'content': 'No, actually multiply 3 and 3!', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'b4fde500-ab20-4797-8577-632061bdd1a3', 'example': False}
--------------------------------------------------
Receiving new event of type: values...
{'content': '', 'additional_kwargs': {'tool_calls': [{'id': 'call_YGnv9gtmRDdAxXu48wTXaupt', 'function': {'arguments': '{"a":3,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 17, 'prompt_tokens': 138, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id':

In [36]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input=None,
    stream_mode="values",
    interrupt_before=["assistant"],
):
    print(f"Receiving new event of type: {chunk.event}...")
    messages = chunk.data.get('messages', [])
    if messages:
        print(messages[-1])
    print("-" * 50)

Receiving new event of type: metadata...
--------------------------------------------------
Receiving new event of type: values...
{'content': '9', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '88ec92d4-0678-4485-a523-a6b88d600681', 'tool_call_id': 'call_YGnv9gtmRDdAxXu48wTXaupt', 'artifact': None, 'status': 'success'}
--------------------------------------------------
Receiving new event of type: values...
{'content': 'The result of multiplying 3 by 3 is 9.', 'additional_kwargs': {'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 14, 'prompt_tokens': 163, 'total_tokens': 177, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BbLTXxMsy8HOrKTQMUqdgjzyFW3gN', 'service_tier'